# Seminar 6. Feature selection & cross-validation pitfalls
Mikhail Belyaev

# Final project 

Two options:
   1. A problem related to your research or innovation activities
   2. A real life problems selected by course instructors (a dataset from UCI or other sources);
    
- You can form groups up to **3** persons. 
- Please, describe the contribution of each person in the Jupyter notebook.

Final projects deliverables:
 - Prepare a presentation using Jupyter notebook and submit it to Canvas
     * Submission Deadline: Thu, 25, 12:30 Moscow time
 - Present your results to course instructors and other students (using the submitted jupyter notebook). 
 - Two possible timeslots for the presentation 
     * Thu, 25
     * Fri, 26
     
Monday, 22 is allocated for consultations.

**Actions required**
 - If you select (1), please prepare a brief description (up to 5-6 sentences) how do you want to use Machine Learning in your project.
 - If you want to form a team, please inform us (you can write a single email, but please add your teammate to the copy of your letter). 
 - If you want to present your work on specific day, include this request in your letter.

**Deadline** - Friday, Oct 19, 12:30 Moscow time. 

Please, send the required information to me (m.belyaev@skoltech.ru) **and** Maxim (m.panov@skoltech.ru). 

If you (or your teammate) don't send us any information, we will assign a problem to you individually.

# Seminar 5 revision

![alt text](http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png "Bias-variance tradeoff")

In [ ]:
from xgboost import XGBClassifier
XGBClassifier()

Which XGBoost parameters are the most import ones? 

How do these parameters affect model complexity?

# Seminar 6

** 6. Features selection**
 - Effective dimensionality reduction;
 - Feature selection approaches: wrappers, filters, embedded methods;
 - Cross-validation pitfals.

# Selection of important features

## Feature selection 

### There are three main groups of feature selection methods:
 - filters
 - wrappers
 - embedded methods 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
% pylab inline

## Filters

- estimate an importance score for each feature
- select K most important one
- there are a lot of different ways to calculate feature importances
- Example: http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection 

### Example 1. Good classification performance, but low statistical score

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

Generate a simple dataset

In [ ]:
def get_dataset1():
    X = np.random.rand(200, 2)
    X[:100, 0] += 1
    X[100:150, 0] += 2
    X[100:, 1] += 0.1
    y = np.concatenate([np.zeros(100), np.ones(100)])
    return X, y

X, y = get_dataset1()

In [ ]:
def plot_data(X, y):
    x_cols = ['x{}'.format(i) for i in range(X.shape[1])]
    df = pd.DataFrame(np.hstack((X, y[:, np.newaxis])), columns=x_cols+['y'])
    if len(x_cols) == 2:
        sns.pairplot(df, hue='y', x_vars='x0', y_vars='x1', size=6)
    else:
        sns.pairplot(df, hue='y', vars=x_cols)
plot_data(X, y)

In [ ]:
# ANOVA
selector = SelectKBest(f_classif, 1)
selector.fit(X, y)
# SelectKBest just selects the specified number of features with the highest scores 
print(X.shape)
X_reduced = selector.transform(X)
print(X_reduced.shape)
# and what about scores?
print(selector.scores_)
# it selects the wrong variable!

In [ ]:
# mutial information is another way to estimate scores ...
selector = SelectKBest(mutual_info_classif, 1)
selector.fit(X, y)
print(selector.scores_)
# and it works in that case

### Example 2. Univariate stats doesn't catch bivariate dependencies

In [ ]:
def get_dataset2(shift=0.2):
    X = np.random.rand(1000, 3)
    X = X[np.abs(X[:, 1] - X[:, 0]) < 0.22]
    X = X[np.abs(X[:, 1] - X[:, 0]) > 0.02]
    y = X[:, 1] > X[:, 0] 
    X[y, 2] += shift
    return X, y
X, y = get_dataset2()

In [ ]:
plot_data(X, y)

In [ ]:
print(SelectKBest(f_classif, 2).fit(X, y).scores_)
print(SelectKBest(mutual_info_classif, 2).fit(X, y).scores_)
# both univariate methods fail

##  Wrappers
 - a greedy alrogithm for feature adding and/or deletion
 - there are a lot of different stratigies (starting points, criteria, etc)
 - an example http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination 

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

svc = SVC(kernel="linear")
rfe = RFE(estimator=svc, n_features_to_select=2, step=1)

In [ ]:
rfe.fit(X, y)
print(rfe.ranking_)
# rank 1 means that these features were selected

X_transformed = rfe.transform(X)
plt.plot(X[:, 0], X[:, 1], '.')

## Embedded 
- features are selected automatically as a part of the learning process 
- an example - linear models with the L1 regularization

![alt text](https://1.bp.blogspot.com/-tXq6Nl2lcNg/V3qzttiZ4sI/AAAAAAAAN_M/6nmjgwydWJUy5Kqt9gFg2Nb12BCTcD4ogCLcB/s1600/LASSO.png  "Embedded feature selection methods")

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l1', C=0.09)
clf.fit(X, y)
print(clf.coef_)

L1-penalty based approaches are a cool class of methods, but in case of correlated variables it can drop relevant features

Let us make the last component fully irrelevant and try a L1-based method again

In [ ]:
X, y = get_dataset2(shift=0)
df = pd.DataFrame(np.hstack((X, y[:, np.newaxis])), columns=['x0', 'x1', 'x2', 'y'])
sns.pairplot(df, hue='y', vars=['x0', 'x1', 'x2'])

In [ ]:
clf = LogisticRegression(penalty='l1', C=0.2)
clf.fit(X, y)
print(clf.coef_)

### Feature selection: a small example

#### TODO: 
 - load an anonimized dataset from dataset3.csv
 - estimate the 10 most important features (using f_classif)
 - perform cross-validation & estimate classification quality

In [ ]:
import pandas as pd
def get_dataset3():
    # insert your code here
    return X, y
X, y = get_dataset3()

In [ ]:
# it's an anonimized dataset with standatized features
print('Data shape is {}'.format(X.shape))
print('Class 0 size is {}'.format(sum(y==0)))
print('Class 1 size is {}'.format(sum(y==1)))
X.head(5)

#### The number of features is too large, so it seems to be a good idea to start with feature selection

In [ ]:
# TODO fit SelectKBest and select 10 best features
# selector = ...

# SelectKBest just selects the specified number of features with the highest scores 

# print(X.shape)
# X_reduced = selector.transform(X)
# print(X_reduced.shape)

#### Now we have a reasonable number of features and can estimate classification accuracy 

In [ ]:
# TODO apply logistic regression to the reduced data set and estimate accuracy using cross_val_score


In [ ]:
# TODO: what if we skip the feature selection step?


#### So, we've obtained a great result using feature selection! Probably, too good to be true ...

Sanity check: randomly shuffle labels

In [ ]:
y_random = y.copy() 
np.random.shuffle(y_random)

In [ ]:
# TODO: estimate accuracy of the pipeline
# as we shuffle the labels, we need to fit the selector again


#### There is a mistake somewhere ...

#### We used *all* available data for feature selection!

### How to do a multistep analysis in a correct way? Use pipeline!

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(SelectKBest(f_classif, 10), 
                      LogisticRegression())
scores = cross_val_score(pipe, X, y_random, scoring='accuracy', cv=5)
print('Accuracy of classification is {:0.2f} +- {:0.2f}'.format(np.mean(scores), np.std(scores)))

scores = cross_val_score(pipe, X, y, scoring='accuracy', cv=5)
print('Accuracy of classification is {:0.2f} +- {:0.2f}'.format(np.mean(scores), np.std(scores)))

### Feature importances: ensembles of trees

Ensembles of trees are usually based on a large set of features, and it's hardly possible to represent such complex models by simple equations. However, there are two other meaningful properties of predictive models which can be used to interpret results. 

- The first useful property is an ability to automatically find important combinations of features and then visualize model predictions as a function of these combinations. The boosted trees models don't allow to create such figures, and the only opportunity is to plot pairwise interactions of the features. 

 - Another desired property of predictive models is an ability to estimate the importance of each feature. Unfortunately, boosted trees as well as other tree-based ensemble methods tend to overestimate importance of actually unimportant features if dataset contains highly correlated features, see: 
 *Auret, L. & Aldrich, C. Empirical comparison of tree ensemble variable importance mea-sures. Chemometrics and Intelligent Laboratory Systems105,157–170 (2011)*

## A sklearn example of embedded feature importances

In [ ]:
import numpy as np
from matplotlib import pylab as plt
%matplotlib inline
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

# Build a classification task using 3 informative features
X, y = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=3,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=0,
                           shuffle=False)

# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

#### TODO: add some correlated features and check the result (keyword n_redundant of make_classification)

In [ ]:
#template to estimate XGB importance
clf = XGBClassifier()
clf.fit(X, y)
importances = clf.feature_importances_
plt.hist(importances);

# Another possible pitfall: selection of hyperparameters & grid search 

In [ ]:
def f_poly(x, coefs):
    summands = [x**(power+1) * coef for power, coef in enumerate(coefs)]
    return np.array(summands).sum(0)

def get_function(coefs=None):
    if coefs is None:
        coefs = [1, -0.5, -1, 0.6]
    return lambda x: f_poly(x, coefs)

def get_dataset4(f, sample_size, noise_std=0.1):
    X = np.random.rand(sample_size, 1) * 2 - 1
    y = f(X)
    y += np.random.randn(*y.shape) * noise_std
    return X, y

def plot_dataset4(f, X=None, y=None, regr=None):
    X_plot = np.linspace(-1, 1, 100)[:, np.newaxis]
    plt.plot(X_plot, f(X_plot), label='True function')
    if X is not None and y is not None:
        plt.plot(X, y, '.r')
    if regr is not None:
        plt.plot(X_plot, regr.predict(X_plot), label='Prediction')
    plt.legend(loc='best')
    plt.ylim([-0.8, 0.6])

#### Lets generate a dataset

In [ ]:
f = get_function()
X, y = get_dataset4(f, 20)
X_test, y_test = get_dataset4(f, 100)
plot_dataset4(f, X, y)

#### We'll use a polinomial model to fit the data and ridge regression to estimate coefficients of the model

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
regr = make_pipeline(PolynomialFeatures(20), Ridge())
regr.fit(X, y)
plot_dataset4(f, X, y, regr)

In [ ]:
from sklearn.metrics.regression import mean_squared_error as mse

def get_errors(regr, X, y):
    y_predicted = regr.predict(X)
    mse(y, y_predicted)**0.5
    return mse(y, y_predicted)**0.5
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))

### Bias-variance tradeoff

![alt text](http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png "Bias-variance tradeoff")

### We can use regularization to control model parameters

In [ ]:
X, y = get_dataset4(f, 20)
regr = make_pipeline(PolynomialFeatures(20), Ridge(1e-9))
regr.fit(X, y)
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))
plot_dataset4(f, X, y, regr)

In [ ]:
X, y = get_dataset4(f, 20)
regr = make_pipeline(PolynomialFeatures(20), Ridge(1e9))
regr.fit(X, y)
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))
plot_dataset4(f, X, y, regr)

In [ ]:
X, y = get_dataset4(f, 20)
regr = make_pipeline(PolynomialFeatures(20), Ridge(0.1))
regr.fit(X, y)
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))
plot_dataset4(f, X, y, regr)

#### So, quality depends on regularization parameter significantly 

### How to select model parameters?

In [ ]:
from sklearn.model_selection import GridSearchCV
# our model is a pipeline, so we have to use a special format to specify parameters
# 
parameters = {'ridge__alpha':10**np.linspace(-5, 5, 21)}
regr = make_pipeline(PolynomialFeatures(20), Ridge())
regr_grid = GridSearchCV(regr, parameters)
regr_grid.fit(X, y)

plot_dataset4(f, X, y, regr_grid)

### But what is we have more than one parameter to adjust?

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
parameters = {'learning_rate': [0.001, 0.005, 0.01, 0.025, 0.05, 0.1],
              'max_depth': range(1, 10),
              'n_estimators': [5, 10, 20, 35, 50, 100],
             }
clf_grid = GridSearchCV(clf, parameters, n_jobs=-1, verbose=True)
X, y = get_dataset3()
clf_grid.fit(X, y)

In [ ]:
best_score_ = clf_grid.best_score_
print(best_score_)
best_params = clf_grid.best_params_
print(best_params)

### An error with a fixed parameters is a random variable

In [ ]:
# TODO: execute this cell multiple times and track function's behavior and the error
X, y = get_dataset4(f, 20)
regr = make_pipeline(PolynomialFeatures(20), Ridge(1e-4))
regr.fit(X, y)
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))
plot_dataset4(f, X, y, regr)

### We can plot distribution of erros

#### for a single model ...

In [ ]:
def get_scores(regr, sample_size, n_repeats):
    scores = []
    for i in range(n_repeats):
        X, y = get_dataset4(f, sample_size)
        regr.fit(X, y)
        scores.append(get_errors(regr, X_test, y_test))
    return scores
regr = make_pipeline(PolynomialFeatures(20), Ridge())
scores = get_scores(regr, sample_size=20, n_repeats=100)
sns.distplot(scores)

#### ... and for several model

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 2, figsize=(7, 7))
sns.despine(left=True)

def plot_hist(alpha, color, ax):
    regr = make_pipeline(PolynomialFeatures(20), Ridge(alpha=alpha))
    scores = get_scores(regr, 20, n_repeats=100)
    sns.distplot(scores, color=color, ax=ax)
    ax.set_title('Regularization is {}'.format(alpha))

plot_hist(alpha=1e-9, color='r', ax=axes[0, 0])
plot_hist(alpha=1e-4, color='g', ax=axes[0, 1])
plot_hist(alpha=1e-1, color='b', ax=axes[1, 0])
plot_hist(alpha=1e9, color='m', ax=axes[1, 1])

### Now check the optimal parameters which were found by GridSearch

In [ ]:
from sklearn.model_selection import KFold
scores = []
X, y = get_dataset3()
for i in range(100):
    clf = GradientBoostingClassifier(**best_params)
    cv = KFold(random_state=i, shuffle=True)
    score = cross_val_score(clf, X, y, cv=cv, n_jobs=-1).mean()
    scores.append(score)

In [ ]:
sns.distplot(scores)
# sns.distplot([best_score_]*100)
plt.plot([best_score_] * 10 , range(10), 'r', label='the score from grid-search')
plt.plot([np.median(scores)] * 10 , range(10), 'g', label='median score')
plt.plot([np.mean(y==0)] * 10 , range(10), 'k', label='naive "always says zero" score')
plt.legend()

##### The grid-search score is better than the highest possible accuracy level. 

### The problem is that GridSearchCV gives you a single number from the whole distribution. Due to a greedy nature of the algorithm, this score is usually too optimistic

## Take-away messages

 - Ideally, use an independent test set 
 - If you use multistep anysis always chain these steps into a single sklearn pipeline
 - As a sanity check, you can feed to your analysis random variables and compare the obrained results with the expected quality of random classification
 - Do not trust GridSearchCV results, always re-check the optimal comination of parameters